In [1]:
from ultralytics import YOLO
import numpy as np
import os
import cv2
from PIL import Image
import pytesseract

In [2]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


In [3]:
#trained yolo model
lp_detect_model = YOLO('yolo_model/best.pt')


In [4]:
#crop the license plate and save as image
def crop_license_plates(img, yolo_model, save_img=True):
    img_name = img.split("\\")[-1]
    
    print(f"Processing {img_name}")
    detected_lps = yolo_model.predict(img)[0]
    license_plates = []
    
    for lp in detected_lps.boxes.data.tolist():
        
        x1, y1, x2, y2, conf, _ = lp
       
        img_array = cv2.imread(img)
        cropped_lp =img_array[int(y1):int(y2), int(x1):int(x2), :]
        license_plates.append(cropped_lp)
        if save_img:
            cv2.imwrite(f"cropped_license_plates/cropped_{img_name}", cropped_lp)
        
        
    return license_plates

In [5]:
def adjust_dpi(image, target_dpi):
    # Convert the NumPy array to Pillow image
    pil_image = Image.fromarray(image)

    # Calculate the scaling factor to achieve the target DPI
    current_dpi = pil_image.info.get('dpi', (72, 72))  # Default DPI if not specified
    scaling_factor = target_dpi / current_dpi[0]  # Assuming X and Y DPI are the same

    # Calculate the new size based on the scaling factor
    new_size = (int(pil_image.width * scaling_factor), int(pil_image.height * scaling_factor))

    # Resize the image using Pillow
    resized_image = pil_image.resize(new_size, resample=Image.LANCZOS)

    # Set the DPI information
    resized_image.info['dpi'] = (target_dpi, target_dpi)

    # Convert the resized image back to a NumPy array
    resized_array = np.array(resized_image)

    return resized_array

In [6]:
def ocr(image):
    custom_config = r'--psm 6 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZÖ0123456789'
    text = pytesseract.image_to_string(image, lang='deu', config=custom_config)
    if text:
        return text.strip("\n")
    return "not detected"

In [34]:
def preprocess_license_plate(license_plate, name, save_img=True):
    kernel = np.ones((3,3),np.uint8)
    # grayscale
    lp_gray = cv2.cvtColor(license_plate, cv2.COLOR_BGR2GRAY)
    # erosion
    lp_gray = cv2.erode(lp_gray, kernel, iterations=1)
    # bilateral filter
    lp_bilateral = cv2.bilateralFilter(lp_gray, 5, 75, 75)
    # binary thresholding
    _, lp_threshold = cv2.threshold(lp_bilateral, 6, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # contours in binary image
    contours, _ = cv2.findContours(lp_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        return None
    # contour with largest area
    max_contour = max(contours, key=cv2.contourArea)

    # Crop the license plate region
    x, y, w, h = cv2.boundingRect(max_contour)
    license_plate_cropped = lp_threshold[y:y + h, x:x + w]

    # Rectify the license plate using perspective transformation
    #rectified_plate = rectify_license_plate(license_plate_cropped)

    if save_img:
        cv2.imwrite(f'thresholded_license_plates/thresh_bil_rectified_{name}', license_plate_cropped)

    return license_plate_cropped

In [24]:
#read characters on licenese plate using OCR  
def read_license_plates(license_plates,name, save_img=True):
    kernel = np.ones((3,3),np.uint8)
    for lp in license_plates:

        #pre processing of the image:
        
        #1.    grayscale the image
       
        lp_gray = cv2.cvtColor(lp, cv2.COLOR_BGR2GRAY)

        #2.     erode the image
        lp_gray = cv2.erode(lp_gray, kernel, iterations=1)

        #3a.     remove noise with median filter
        lp_median = cv2.medianBlur(lp_gray,5)
        
        #3b.     remove noise with bilateral filter
        lp_bilateral = cv2.bilateralFilter(lp_gray, 5, 75, 75)

        #4a.    canny edge detection 
        lp_canny = cv2.Canny(lp_median, 100, 200)

        #4b.     binary thresholding 
        _, lp_threshold = cv2.threshold(lp_median, 6, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        _, lp_threshold2 = cv2.threshold(lp_bilateral, 6, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        if save_img:
            cv2.imwrite(f'thresholded_license_plates/thresh_median_{name}', lp_threshold)
            cv2.imwrite(f'thresholded_license_plates/thresh_bil_{name}', lp_threshold2)
            cv2.imwrite(f'thresholded_license_plates/canny_{name}', lp_canny)

        return lp_threshold, lp_threshold2, lp_canny

In [38]:
path_to_cam_frames = 'cam_frames'
res = []
for f in os.listdir(path_to_cam_frames):
    if f.endswith('.jpg') or f.endswith('.png'):
        img = os.path.join(path_to_cam_frames, f)

        test_lps = crop_license_plates(img, lp_detect_model)
        print(len(test_lps))
        median, bilateral, canny = read_license_plates(test_lps,f)
        re_cropped = preprocess_license_plate(test_lps[0], f)
        ocr_recrop = ocr(re_cropped)
        ocr_median = ocr(median)
        ocr_bilateral = ocr(bilateral)
        ocr_canny = ocr(canny)
        res.append([f[:-4], ocr_median, ocr_bilateral, ocr_canny, ocr_recrop])


Processing image_1.jpg


image 1/1 e:\Users\belau\Documents\BV-ML-CV-Praktikum\cam_frames\image_1.jpg: 512x640 1 license_plate, 217.2ms
Speed: 2.0ms preprocess, 217.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


1


Processing image_2.jpg


image 1/1 e:\Users\belau\Documents\BV-ML-CV-Praktikum\cam_frames\image_2.jpg: 512x640 1 license_plate, 168.9ms
Speed: 2.0ms preprocess, 168.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


...


image 1/1 e:\Users\belau\Documents\BV-ML-CV-Praktikum\cam_frames\image_4.jpg: 512x640 1 license_plate, 160.7ms
Speed: 1.0ms preprocess, 160.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


Processing image_4.jpg
1


Processing image_5.jpg


image 1/1 e:\Users\belau\Documents\BV-ML-CV-Praktikum\cam_frames\image_5.jpg: 512x640 1 license_plate, 182.2ms
Speed: 1.0ms preprocess, 182.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


1


In [37]:
import pandas as pd

res = pd.DataFrame(res)
print(res)
res.to_csv("ocr_results.csv", index=False, header=["Image name", "Canny", "Bilateral + Tresh","Median + Tresh", "recropped"])

         0             1           2          3         4
0  image_1     SHGALF2O6  ISHGRLF2O6  SWGRUGZUE  SHGLF2O6
1  image_2      SHGLF2O6   ISHGLF2O6   SE8LEZOA  SHGLF2O6
2  image_3  not detected    SHGLF2O6   SSCLEATS  SHGLF2O6
3  image_4     SHGRLF2O6    SHGLF2O6  SHESUR2UE  SHGLF2O6
4  image_5     SHGTLF286    SHGLF2O6  SHESUE2UA  SHGLF2O6
